In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# ✅ 프로젝트 경로 설정
import os

project_path = "/content/drive/MyDrive/Colab Notebooks/youtube/youtube_interest_classifier"
os.makedirs(project_path, exist_ok=True)
os.chdir(project_path)

In [3]:
pwd

'/content/drive/MyDrive/Colab Notebooks/youtube/youtube_interest_classifier'

In [4]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import joblib

In [8]:
# ✅ 로딩
model = AutoModelForSequenceClassification.from_pretrained(os.path.join(project_path, "model"))
tokenizer = AutoTokenizer.from_pretrained(os.path.join(project_path, "model"))
le = joblib.load(os.path.join(project_path, "label_encoder.pkl"))

In [9]:
def preprocess_input(channel_name: str, video_title: str) -> str:
    return f"YouTuber: {channel_name} [SEP] Title: {video_title}"

In [10]:
def predict_category_batch(df):
    # 1) 전처리
    df["input_text"] = df.apply(
        lambda row: preprocess_input(row["channel_name"], row["video_title"]),
        axis=1
    )

    # 2) Tokenization
    inputs = tokenizer(
        list(df["input_text"]),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=64
    )

    # 3) 모델 예측
    with torch.no_grad():
        logits = model(**inputs).logits
        preds = torch.argmax(logits, dim=1).tolist()
        categories = le.inverse_transform(preds)

    df["predicted_category"] = categories
    return df

In [18]:
user_test_df = pd.DataFrame({
    "channel_name": ["뜬뜬 DdeunDdeun", "PONY Syndrome", "삼프로TV", "소비더머니", "JTBC News", "한경 코리아마켓", "해니 haeni beauty", "오분순삭", "꿘나"],
    "video_title": [
        "[EN/JP]양반들의 마을에 게판의 등장이라...⭐ 재밌어지겠네 | 깡촌캉스 EP.2",
        "모카무스 메이크업 ☕ Mocha Mousse Makeup",
        "미국 금리 인상, 한국 시장에 어떤 영향이 있을까?",
        "찐 흙수저 재벌! 웅진그룹의 성장과 해체, 그리고 재도전 / 소비더머니",
        "[LIVE] 승복 선언 나오나.. 이 시각 김문수 후보 기자회견 [이슈현장]",
        "대선 D-1, 갈팡질팡 증시 | 파랗게 질린 철강주 [6월2일 #퇴근요정]",
        "넙데데 얼굴 커버하는법 레이, 닝닝 메이크업 Tip!!",
        "[무한도전] 왔다 내 도파민🤑 돈으로도 못 사는 무한도전 표 명품 티키타카 모음.zip | 무한도전⏱오분순삭 MBC070915방송",
        "982147번 물어보신 메이크업"
    ]
})

result_df = predict_category_batch(user_test_df)
print(result_df[["channel_name", "video_title", "predicted_category"]])

      channel_name                                        video_title  \
0    뜬뜬 DdeunDdeun    [EN/JP]양반들의 마을에 게판의 등장이라...⭐ 재밌어지겠네 | 깡촌캉스 EP.2   
1    PONY Syndrome                    모카무스 메이크업 ☕ Mocha Mousse Makeup   
2            삼프로TV                       미국 금리 인상, 한국 시장에 어떤 영향이 있을까?   
3            소비더머니            찐 흙수저 재벌! 웅진그룹의 성장과 해체, 그리고 재도전 / 소비더머니   
4        JTBC News         [LIVE] 승복 선언 나오나.. 이 시각 김문수 후보 기자회견 [이슈현장]   
5         한경 코리아마켓          대선 D-1, 갈팡질팡 증시 | 파랗게 질린 철강주 [6월2일 #퇴근요정]   
6  해니 haeni beauty                     넙데데 얼굴 커버하는법 레이, 닝닝 메이크업 Tip!!   
7             오분순삭  [무한도전] 왔다 내 도파민🤑 돈으로도 못 사는 무한도전 표 명품 티키타카 모음.z...   
8               꿘나                                  982147번 물어보신 메이크업   

  predicted_category  
0      Howto , Style  
1      Howto , Style  
2    News , Politics  
3      Entertainment  
4    News , Politics  
5    News , Politics  
6      Howto , Style  
7     People , Blogs  
8      Howto , Style  
